In [7]:
import pandas as pd
#import utils from parent folder
# Determine the absolute path to the Subfolder2 directory
import sys
import os
utils_dir = os.path.abspath(os.path.join(os.getcwd(),  '..', '2023'))
import subprocess
import pandas as pd
import json
# Add the Subfolder2 directory to the system path
sys.path.append(utils_dir)

import numpy as np

from utils import *

In [2]:
def get_wildlife_data_sql():
    engine = get_conn('sde_tabular')
    # get BMP Status data as dataframe from BMP SQL Database
    with engine.begin() as conn:
        # create dataframe from sql query
        df = pd.read_sql("SELECT * FROM sde_tabular.SDE.ThresholdEvaluation_Wildlife", conn)
    return df

    #return the data
    return df

In [3]:
wildlife_data = get_wildlife_data_sql()
bald_eagle_individuals = wildlife_data[wildlife_data['Wildlife_Species'] == 'Bald Eagle - winter']
bald_eagle_nesting = wildlife_data[wildlife_data['Wildlife_Species'] == 'Bald Eagle - nesting']
osprey_nests = wildlife_data[wildlife_data['Wildlife_Species'] == 'Osprey']
peregrine_falcon_nests = wildlife_data[wildlife_data['Wildlife_Species'] == 'Peregrine Falcon']


In [27]:
def generate_csv(df,filename, year_start = None, year_end= None):
    df = df.sort_values(by=['Year']).reset_index(drop=True)
    if year_start is not None:
        df = df[df['Year'] >= year_start]
    if year_end is not None:
        df = df[df['Year'] <= year_end]
    df_formatted = pd.DataFrame()
    df_formatted['Observed.t']= df['Total']
    df_formatted['Time.t']= df.index
    df_formatted.to_csv(filename, index=False)
    return df_formatted

In [28]:
# generate csv files for each species
generate_csv(bald_eagle_individuals, 'bald_eagle_individuals.csv')
#generate_csv(bald_eagle_nesting, 'bald_eagle_nesting.csv')
generate_csv(osprey_nests, 'osprey_nests.csv')
generate_csv(peregrine_falcon_nests, 'peregrine_falcon_nests.csv')

,Observed.t,Time.t
0,0.0,0
1,1.0,1
2,1.0,2
3,2.0,3
4,2.0,4
5,2.0,5
6,3.0,6
7,4.0,7
8,4.0,8
9,4.0,9


In [29]:
generate_csv(osprey_nests, 'osprey_nests_post_2002.csv', year_start=2002)

,Observed.t,Time.t
5,21.0,5
6,29.0,6
7,28.0,7
8,22.0,8
9,24.0,9
10,24.0,10
11,22.0,11
12,24.0,12
13,26.0,13
14,26.0,14


In [41]:
test_osprey = generate_csv(osprey_nests, r'C:\Users\amcclary\Documents\GitHub\ThresholdEvaluation\Scripts\Analysis\osprey.csv')

In [10]:
import subprocess
import pandas as pd
import json

# Define your inputs
csv_file = r'C:/Users/amcclary/Documents/GitHub/ThresholdEvaluation/Scripts/Wildlife/osprey.csv'
output_dir = r'C:/Users/amcclary/Documents/GitHub/ThresholdEvaluation/Scripts/Wildlife'
rscript = r'C:/Users/amcclary/Documents/GitHub/ThresholdEvaluation/Scripts/Wildlife/Population_Trends.R'

# Call the R script using subprocess
result = subprocess.run(
    ["Rscript", rscript, csv_file, output_dir], 
    capture_output=True, 
    text=True
)

# Check for errors
if result.returncode != 0:
    print("Error:", result.stderr)
else:
    # Parse JSON output from the R script
    results = json.loads(result.stdout)
    
    # Convert the results to a DataFrame
    output_df = pd.DataFrame(list(results.items()), columns=['Parameter', 'Value'])
    
    # Display the DataFrame
    print(output_df)


   Parameter                                              Value
0       EGOE  {'mu': 0.0131, 'x0': 2.9754, 'ssq': 0, 'tsq': ...
1       EGPN  {'mu': 0.0215, 'x0': 2.4849, 'ssq': 0.0272, 't...
2    EGSS_ML  {'mu': 0.019, 'ssq': 0.0117, 'tsq': 0.0069, 'x...
3  EGSS_REML  {'mu': [[0.0192]], 'ssq': 0.0138, 'tsq': 0.006...


In [34]:
# Next step, run this for each dataframe and then create plots with the predicted line and the observed data
# access the mu value from the output_df 
# Extract 'mu' values from the 'data' column
df = output_df.copy()
df['mu'] = df['Value'].apply(lambda x: x.get('mu', None))  # Adjust key as needed
df['ssq'] = df['Value'].apply(lambda x: x.get('ssq', None))  # Adjust key as needed
df['tsq'] = df['Value'].apply(lambda x: x.get('tsq', None))  # Adjust key as needed
df['growth_rate'] = np.exp(df['mu'])
df['Start_Year'] = 2006
df['End_Year'] = 2023
mu = df.loc[df['Parameter']=='EGPN', 'mu'].values[0]
# Display the DataFrame
# inverse natural log transformation
mu = np.exp(mu)
#print(mu)
osprey_df = osprey_nests.copy()
# Initialize the starting value
osprey_df['adjusted_value'] = osprey_df['Total']  # Start with the original value
osprey_df = osprey_df.reset_index(drop=True)
# Apply the increase factor cumulatively
print(osprey_df.loc[1, 'adjusted_value'])

for i in range(1, len(osprey_df)):
     osprey_df.loc[i, 'adjusted_value'] = osprey_df.loc[i-1, 'adjusted_value'] * mu
osprey_df['Predicted'] = osprey_df['adjusted_value']
osprey_df['Observed'] = osprey_df['Total']

17.0


In [25]:
def generate_trend_estimate(df, species, csv_file, output_dir, rscript, year_start = None, year_end= None):
    df_species = df.loc[df['Wildlife_Species'] == species]
    df_species = df_species.sort_values(by=['Year']).reset_index(drop=True)
    if year_start is not None:
        df_species = df_species[df_species['Year'] >= year_start]
    if year_end is not None:
        df_species = df_species[df_species['Year'] <= year_end]
    max_year = df_species['Year'].max()
    min_year = df_species['Year'].min()
    df_formatted = pd.DataFrame()
    df_formatted['Observed.t']= df_species['Total']
    df_formatted['Time.t']= df_species.index
    df_formatted.to_csv(csv_file, index=False)
    result = subprocess.run(
    ["Rscript", rscript, csv_file, output_dir], 
    capture_output=True, 
    text=True
    )

    # Check for errors
    if result.returncode != 0:
        print("Error:", result.stderr)
    else:
        # Parse JSON output from the R script
        results = json.loads(result.stdout)
        
        # Convert the results to a DataFrame
        output_df = pd.DataFrame(list(results.items()), columns=['Parameter', 'Value'])
    
    df = output_df.copy()
    df['mu'] = df['Value'].apply(lambda x: x.get('mu', None))  # Adjust key as needed
    df['ssq'] = df['Value'].apply(lambda x: x.get('ssq', None))  # Adjust key as needed
    df['tsq'] = df['Value'].apply(lambda x: x.get('tsq', None))  # Adjust key as needed
    df['lower'] = df['Value'].apply(lambda x: x.get('ci_mu', {}).get('lower', None)) 
    df['upper'] = df['Value'].apply(lambda x: x.get('ci_mu', {}).get('upper', None))
    
    df['mu']=pd.to_numeric(df['mu'], errors='coerce')
    df['lower']=pd.to_numeric(df['lower'], errors='coerce')
    df['upper']=pd.to_numeric(df['upper'], errors='coerce')
    df['growth_rate'] = np.exp(df['mu'])
    df['growth_rate_lower'] = np.exp(df['lower'])
    df['growth_rate_upper'] = np.exp(df['upper'])
    df['growth_percent'] = (df['growth_rate'] - 1) * 100
    df['growth_percent_lower'] = (df['growth_rate_lower'] - 1) * 100
    df['growth_percent_upper'] = (df['growth_rate_upper'] - 1) * 100
    df['Start_Year'] = min_year
    df['End_Year'] = max_year
    df.drop(columns=['Value'], inplace=True)
    df['Species'] = species
    return df


In [43]:
osprey_trend = generate_trend_estimate(wildlife_data, 'Osprey', 
                                       r'C:\Users\amcclary\Documents\GitHub\ThresholdEvaluation\Scripts\Analysis\osprey.csv', 
                                       r'C:\Users\amcclary\Documents\GitHub\ThresholdEvaluation\Scripts\Analysis', 
                                       r'C:\Users\amcclary\Documents\GitHub\ThresholdEvaluation\Scripts\Analysis\Wildlife\Population_Trends.R')

In [26]:
csv_file = r'C:/Users/amcclary/Documents/GitHub/ThresholdEvaluation/Scripts/Wildlife/input.csv'
output_dir = r'C:/Users/amcclary/Documents/GitHub/ThresholdEvaluation/Scripts/Wildlife'
rscript = r'C:/Users/amcclary/Documents/GitHub/ThresholdEvaluation/Scripts/Wildlife/Population_Trends.R'
osprey_estimates_all_years = generate_trend_estimate(wildlife_data, 'Osprey', csv_file, output_dir, rscript)
print (osprey_estimates_all_years)
bald_eagle_individuals_estimates_all_years = generate_trend_estimate(wildlife_data, 'Bald Eagle - winter', csv_file, output_dir, rscript)
print (bald_eagle_individuals_estimates_all_years)
peregrine_falcon_nests_estimates_no_year_0 = generate_trend_estimate(wildlife_data, 'Peregrine Falcon', csv_file, output_dir, rscript, year_start = 2009, year_end= 2023)
osprey_estimates_post_2002 = generate_trend_estimate(wildlife_data, 'Osprey', csv_file, output_dir, rscript, year_start = 2003)
osprey_test = generate_trend_estimate(wildlife_data, 'Osprey', csv_file, output_dir, rscript, year_end= 2015)
all_estimates = pd.concat([osprey_estimates_all_years, bald_eagle_individuals_estimates_all_years, peregrine_falcon_nests_estimates_no_year_0, osprey_estimates_post_2002, osprey_test])

   Parameter      mu     ssq     tsq   lower   upper  growth_rate  \
0       EGOE  0.0131  0.0000  0.0290 -0.0354  0.0615     1.013186   
1       EGPN  0.0215  0.0272  0.0000 -0.3629  0.4059     1.021733   
2    EGSS_ML  0.0190  0.0117  0.0069 -0.0267  0.0647     1.019182   
3  EGSS_REML     NaN  0.0138  0.0067     NaN     NaN          NaN   

   growth_rate_lower  growth_rate_upper  growth_percent  growth_percent_lower  \
0           0.965219           1.063430        1.318618             -3.478075   
1           0.695656           1.500652        2.173279            -30.434400   
2           0.973653           1.066839        1.918165             -2.634671   
3                NaN                NaN             NaN                   NaN   

   growth_percent_upper  Start_Year  End_Year Species  
0              6.343050      1997.0    2023.0  Osprey  
1             50.065248      1997.0    2023.0  Osprey  
2              6.683892      1997.0    2023.0  Osprey  
3                   NaN 

In [19]:
all_estimates['mu'] = pd.to_numeric(all_estimates['mu'], errors='coerce')

# Apply the exponential function to the 'mu' column
all_estimates['growth_rate'] = np.exp(all_estimates['mu'])

In [18]:
print(all_estimates['mu'].dtype)
print(all_estimates['mu'])


object
0     0.0131
1     0.0215
2      0.019
0     0.0496
1     0.0439
2     0.0496
0     0.1081
1      0.099
2   -11.5401
0    -0.0019
1    -0.0161
2    -0.0019
Name: mu, dtype: object


In [54]:
all_estimates.to_csv(r'C:\Users\amcclary\Documents\GitHub\ThresholdEvaluation\Scripts\Analysis\population_estimates.csv', index=False)

In [35]:
#plot the observed and predicted data with year on the x axis using plotly
import plotly.express as px
fig = px.line(osprey_df, x='Year', y=['Observed', 'Predicted'], title='Osprey Population Trends')
fig.show()